# Chatbot Agent

This notebook demonstrates the implementation of a chatbot agent using transformers.


In [ ]:
!pip install tensorflow
!pip install transformers
!pip install datasets
!pip install torch




In [ ]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

class CustomerSupportAgent:
    """
    A customer support AI agent that uses a language model to interact with users,
    providing responses based on user queries. The agent handles perception,
    decision-making, and action-taking to simulate a conversational support experience.
    """
    def __init__(self, model_name="EleutherAI/gpt-neo-125M"):
        """
        Initializes the CustomerSupportAgent with a specified language model.

        Parameters:
        - model_name (str): The name of the pre-trained language model to use.
        """
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        self.chatbot = pipeline("text-generation", model=self.model, tokenizer=self.tokenizer)
        self.conversation_history = []

    def perceive(self, user_input):
        """
        Processes the user's input and updates the conversation history.

        Parameters:
        - user_input (str): The user's input message.

        Returns:
        - str: The processed user input, converted to lowercase.
        """
        self.conversation_history.append(f"User: {user_input}")
        return user_input.lower()

    def decide(self, user_input):
        """
        Generates a response based on the user's input and conversation history.

        Parameters:
        - user_input (str): The processed user input.

        Returns:
        - str: The generated response from the language model.
        """
        prompt = "\n".join(self.conversation_history) + "\nAssistant:"
        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512)
        output = self.model.generate(**inputs, max_new_tokens=50, do_sample=True, temperature=0.7)
        return self.tokenizer.decode(output[0], skip_special_tokens=True)

    def act(self, response):
        """
        Finalizes the response, appends it to the conversation history, and returns it.

        Parameters:
        - response (str): The generated response from the language model.

        Returns:
        - str: The final response to be provided to the user.
        """
        self.conversation_history.append(f"Assistant: {response}")
        return response

    def run(self):
        """
        Runs the interaction loop, allowing continuous conversation with the user.
        The loop terminates when the user inputs 'exit' or 'quit'.
        """
        print("Assistant: Hello! How can I assist you today?")
        while True:
            user_input = input("You: ")
            if user_input.lower() in ["exit", "quit"]:
                print("Assistant: Goodbye!")
                break
            perceived_input = self.perceive(user_input)
            response = self.decide(perceived_input)
            print(f"Assistant: {self.act(response)}")

# Running the AI agent
agent = CustomerSupportAgent()
agent.run()


